# Exploring the Data

In this file we begin our initial exploration and preparation of the dataset. The goal is to put together a dataset that we can analyze to provide advice to a potential future movie studio that our stakeholder Microsoft is wanting to get off the ground.

## Importing the Data

The data are contained in the zippedData directory of this repo and will need to be accessed with an unzip function. 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import gzip
from io import StringIO
%matplotlib inline

In [2]:
#Defining the unzip funciton

def unZip_csv(file_location): 
    file = gzip.open(file_location, 'rb')
    content = file.read()
    file.close()
    content_str = str(content,'utf-8')
    content_data = StringIO(content_str)
    
    
    #its important to remember pd.read_csv will also read tsv using an 
    #additional key so we need to direct input to the correct pd.read
    
    if '.tsv' in file_location:
        return pd.read_csv(content_data, sep='\t')
    else:
        return pd.read_csv(content_data)

We decided to start with a data set that contains the production costs as well as revenue data to draw more meaningful conlusions about profit and return on investment. That data was contained in the `'zippedData/tn.movie_budgets.csv.gz'` file

In [3]:
df = unZip_csv('zippedData/tn.movie_budgets.csv.gz')

### Exploring the DataFrame

At first glance this data contains no null values and relativley detailed revenue data

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


In [5]:
df.isna().sum()

id                   0
release_date         0
movie                0
production_budget    0
domestic_gross       0
worldwide_gross      0
dtype: int64

In [6]:
df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


It looks like this file contained information about the release data and revenue info, but there are a few other bits of info that we will need to make useful reccomendations. We'll want information about the genre and ratings, which we can get from:

`'zippedData/imdb.title.basics.csv.gz'` and `'zippedData/tmdb.movies.csv.gz'`.

In [7]:
imdb_title = unZip_csv('zippedData/imdb.title.basics.csv.gz')

imdb_movies = unZip_csv('zippedData/tmdb.movies.csv.gz')

First let's look at the columnsi n `imdb_title` and match up the name of the column we're going to merge on.

In [8]:
#need a matching column name to merge on
imdb_title['movie'] = imdb_title['primary_title']
df = df.merge(imdb_title[['movie', 'genres']], on='movie')

imdb_movies['movie'] = imdb_movies['title']
df = df.merge(imdb_movies[['movie','vote_average','vote_count']])

df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,genres,vote_average,vote_count
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",Horror,7.4,18676
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875","Action,Adventure,Fantasy",6.4,8571
2,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963","Action,Adventure,Sci-Fi",7.3,13457
3,7,"Apr 27, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,048,134,200","Action,Adventure,Sci-Fi",8.3,13948
4,9,"Nov 17, 2017",Justice League,"$300,000,000","$229,024,295","$655,945,209","Action,Adventure,Fantasy",6.2,7510


Now we have all the data we need to make our analysis conciseley in one DataFrame. Our next task will be reformatting the data to give us a much easier time using the data.

## Cleaning the Data

In this section we will apply techniques to reformat and clarify the data while preserving the intended values.

### Let's reformat the data types

We're going to want to take a look at decoding the foreign gross revenue and net profit figures from the existing data for our analysis. But before we can calculate any new figures we have to reformat the values to their propert data type.

In [9]:
#using a helper function to map int values onto the dollar value strings
def toInt (df_series):
    return df_series.map(lambda x: int(x[1:].replace(",","")))

df['production_budget'] = toInt(df['production_budget'])
df['domestic_gross'] = toInt(df['domestic_gross'])
df['worldwide_gross'] = toInt(df['worldwide_gross'])

Next we're going to want the date column formatted as a pd.datetime object so we can implement that data when plotting. This is also a great time to drop off any redundant or useless columns we can identify.

In [10]:
#changing date format
df['release_date'] = pd.to_datetime(df['release_date'])

#dropping redundant index
df.drop('id',axis=1,inplace=True)

#clean off the column headers and object data in the movie column
df.columns = df.columns.str.strip()
df['movie'] = df['movie'].str.strip()

In [11]:
df.head(2)

,release_date,movie,production_budget,domestic_gross,worldwide_gross,genres,vote_average,vote_count
0,2009-12-18,Avatar,425000000,760507625,2776345279,Horror,7.4,18676
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,"Action,Adventure,Fantasy",6.4,8571


#### Unpacking the genres column

The data in this column will be paramount for analyzing trends in consumer preference, but its not of any use to us in its current format. Lets see if we can get those strings separated into lists

In [51]:
df['genres'].str.split(',')

0                           [Horror]
1       [Action, Adventure, Fantasy]
2        [Action, Adventure, Sci-Fi]
3        [Action, Adventure, Sci-Fi]
4       [Action, Adventure, Fantasy]
                    ...             
4545    [Adventure, Horror, Mystery]
4546     [Horror, Mystery, Thriller]
4547               [Comedy, Romance]
4548              [Sci-Fi, Thriller]
4549                 [Comedy, Drama]
Name: genres, Length: 4550, dtype: object

### Re-working the columns

For ease of use we want to create a new column for profit and foreign gross revenue so we can reference those values on the fly.

In [13]:
df['profit'] = df['worldwide_gross'] - df['production_budget']
df['foreignGross'] = df['worldwide_gross'] - df['domestic_gross']
#Note that we're changing the naming convention here

Suppose we want to have a measure of return on investment, which is an easily understand able and ubiquidous measurement used to see the relationship between investment and the bottom line.

    ROI = Net Income / Cost of Investment
    
Or in our case

    ROI = Profit Before Tax / Production Budget

In [14]:
df['%roi'] = df['profit'] / df['production_budget'] * 100
df['%roi'] = df['%roi'].round(decimals=2)

In [15]:
df.head()

,release_date,movie,production_budget,domestic_gross,worldwide_gross,genres,vote_average,vote_count,profit,foreignGross,%roi
0,2009-12-18,Avatar,425000000,760507625,2776345279,Horror,7.4,18676,2351345279,2015837654,553.26
1,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,"Action,Adventure,Fantasy",6.4,8571,635063875,804600000,154.67
2,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,"Action,Adventure,Sci-Fi",7.3,13457,1072413963,944008095,324.38
3,2018-04-27,Avengers: Infinity War,300000000,678815482,2048134200,"Action,Adventure,Sci-Fi",8.3,13948,1748134200,1369318718,582.71
4,2017-11-17,Justice League,300000000,229024295,655945209,"Action,Adventure,Fantasy",6.2,7510,355945209,426920914,118.65


Beautful! Now we should look at reorganizing these columns in a way that makes more sense. This is subjective but we think the profit should be followed by the revenues then followed up again by production costs. Our ROI encodes production value into more digestible metrics but we will still have to column to reference if needed.

In [16]:
column_map = [
    'release_date'
    ,'movie'
    ,'profit'
    ,'%roi'
    ,'worldwide_gross'
    ,'domestic_gross'
    ,'foreignGross'
    ,'production_budget'
    ,'vote_average'
    ,'vote_count'
    ,'genres'


]

new_names = [
    'releaseDate'
    ,'title'
    ,'profit'
    ,'%roi'
    ,'grossRevenue'
    ,'domesticGross'
    ,'foreignGross'
    ,'productionBudget'
    ,'imdbRating'
    ,'imdbRatingCount'
    ,'genres'
]

mapper = {k:v for k,v in zip(column_map,new_names)}

df = df.reindex(columns=column_map,copy=False)
df = df.rename(columns=mapper)

In [17]:
df.columns

Index(['releaseDate', 'title', 'profit', '%roi', 'grossRevenue',
       'domesticGross', 'foreignGross', 'productionBudget', 'imdbRating',
       'imdbRatingCount', 'genres'],
      dtype='object')